In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
from matplotlib.patches import Polygon 
from scipy.io import loadmat
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from matplotlib.text import Text

In [ ]:
path = os.path.abspath('../../../1_Simulation_Results/numpy_files/')

colormap_data = loadmat('../cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

## Gaussian HS

In [ ]:
LIQ   = np.load(path+'/Gaussian/TDB/LIQ.npy')
temp  = np.load(path+'/Gaussian/TDB/temp.npy')
timeS = np.load(path+'/Gaussian/TDB/time.npy')
time_steps =  timeS.shape[0]

## FlatTop HS

In [ ]:
LIQ   = np.load(path+'/FlatTop/TDB/LIQ.npy')
temp  = np.load(path+'/FlatTop/TDB/temp.npy')
timeS = np.load(path+'/FlatTop/TDB/time.npy')
time_steps =  timeS.shape[0]

## Ring HS

In [ ]:
LIQ   = np.load(path+'/Ring/TDB/LIQ.npy')
temp  = np.load(path+'/Ring/TDB/temp.npy')
timeS = np.load(path+'/Ring/TDB/time.npy')
time_steps =  timeS.shape[0]

## Bessel HS

In [ ]:
LIQ   = np.load(path+'/Bessel/TDB/LIQ.npy')
temp  = np.load(path+'/Bessel/TDB/temp.npy')
timeS = np.load(path+'/Bessel/TDB/time.npy')
time_steps =  timeS.shape[0]

### Fitting the gap

In [ ]:
end_step = max((i for i, v in enumerate(list(timeS)) if v <= 25), key=lambda i: list(timeS)[i])
end_step

In [ ]:
def leading_front(array, condition, end_time_step):
    condition = condition
    end_step = end_time_step
    leading_front = []
    for t in range(end_step):
        A = array[0:1,:,t][0]
        for i in range(len(A)-1,0,-1):
            if A[i]>=condition:
                # print(i)
                break
        leading_front.append(i)
    return leading_front

In [ ]:
leading_temperature = leading_front(array=temp[:,:,:end_step], condition=1337, end_time_step=end_step)
leading_LIQ = leading_front(array=LIQ[:,:,:end_step], condition=0.5, end_time_step= end_step)
time_duration = np.arange(end_step)
time_lead_lag =  timeS[:end_step]

In [ ]:
def fit_real_time(data, order):
    position, time_lead_lag = data
    coefficients_real = np.polyfit(position, time_lead_lag, order)
    fitted_real_time = np.poly1d(coefficients_real)
    return fitted_real_time
    
common_position = leading_LIQ.index(leading_temperature[1]) # = 36 ## => comes from nearest value of starting point of leading temp i.e. => leading_temperature[1]
phase_fit_real_time = fit_real_time([leading_LIQ[1:], time_lead_lag[1:]], 15)
temp_fit_real_time = fit_real_time([leading_temperature[1:], time_lead_lag[1:]], 15)

In [ ]:
lag_time = []
for t in range(end_step):
    lag_time.append(phase_fit_real_time(leading_LIQ[t]) - temp_fit_real_time(leading_LIQ[t]))

In [ ]:
t_step = 400

fig, (ax1) = plt.subplots(1, 1, figsize=(12,6), frameon=False)
ax1.set_facecolor((0, 0, 0, 0))

ax1.plot(leading_temperature[1:t_step], time_lead_lag[1:t_step], marker='o', c='green',markersize=1, linewidth=6, label =r'$\tau$')
ax1.plot(leading_LIQ[1:t_step], time_lead_lag[1:t_step], marker='.', c='red',markersize=1,linewidth=6, label =r'$\mathfrak{D}$')

ax1.set_ylabel("Time Duration [s]",  weight = 'bold', size=20,  color='red')
ax1.set_xlabel(r"Horizontal position $\mathbf{\mu}$m $\longrightarrow$", weight='bold', size=25, color='m', labelpad=1, ha='center')
ax1.set_xticks([50, 200, 350], labels=['125', '500', '875'], size=25, color='m', weight='bold')
ax1.yaxis.set_tick_params(labelcolor='green', labelsize=25, pad=3,  direction='inout', length=20, width=5, color='k',)
ax1.xaxis.set_tick_params(labelcolor='m', labelsize=25, pad=3,  direction='inout', length=20, width=5, color='k',)
for label in ax1.get_yticklabels(): label.set_fontweight('bold')

if t_step>=common_position:
    ax1.hlines(y=phase_fit_real_time(leading_LIQ[t_step]), xmax=leading_LIQ[t_step], xmin=leading_LIQ[t_step]-55, linewidth=3, linestyle=':', color='red')
    ax1.hlines(y=temp_fit_real_time(leading_LIQ[t_step]), xmax=leading_LIQ[t_step], xmin=leading_LIQ[t_step]-55, linewidth=3, linestyle=':', color='green')
    ax1.text(x=leading_LIQ[t_step]-72, y=np.mean([phase_fit_real_time(leading_LIQ[t_step]), temp_fit_real_time(leading_LIQ[t_step])]), s=r"$\Delta$t", color='blue', fontsize=30, rotation=90, weight='bold')
    ax1.annotate(text=' ', xy=(leading_LIQ[t_step]-55+1, temp_fit_real_time(leading_LIQ[t_step])-0.5), xytext=(leading_LIQ[t_step]-55,phase_fit_real_time(leading_LIQ[t_step])+0.8), arrowprops=dict(arrowstyle='<->', color='blue'))

ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
ax1.set_ylim([-1,28])
ax1.set_xlim([0,420])
legend_1 = ax1.legend(loc=(0.02,0.65), prop={'weight':'bold', 'size':'40',}, handletextpad=0.1, handlelength=1.0,   frameon=False).get_frame().set_facecolor('none')

ax2 = ax1.twinx()
if t_step>common_position:
    ax2.plot(leading_LIQ[common_position:t_step],lag_time[common_position:t_step],   c='blue', linewidth=6, label =r'$\Delta t_{x}$') # positional_timelag

legend_2 = ax2.legend(loc=(0.80,0.01), prop={'weight':'bold', 'size':'40',}, handletextpad=0.1, handlelength=1.0, frameon=False).get_frame().set_facecolor('none')

ax2.set_ylabel(r"$\mathbf{\delta t_{pos}}$ [s]",  weight = 'bold', size=30, labelpad=10, color='blue')
ax2.yaxis.set_tick_params(labelcolor='blue', labelsize=30,direction='inout', length=20, width=5, color='k',)
ax2.yaxis.set_major_locator(plt.MaxNLocator(4))
for label in ax2.get_yticklabels(): label.set_fontweight('bold')
ax2.set_ylim(1.25,1.75)
ax2.set_xlim([0,420])

ax1.spines[:].set_linewidth(4)


plt.tight_layout()

# plt.savefig('../../figures/LLD_GHS.png',bbox_inches='tight', dpi=800)
# plt.savefig('../../figures/jpg/LLD_GHS.jpg',bbox_inches='tight',)
# plt.savefig(f'../../../../0_GitHub_hostings/Digital_Additive_Manufacturing//4_Manuscript_Figures/Temperature_FT_{timeS_FT[t_step]:.0f}s.png',bbox_inches='tight', dpi=800)
# plt.savefig(f'../../../../0_GitHub_hostings/Digital_Additive_Manufacturing//4_Manuscript_Figures/jpg/Temperature_FT_{timeS_FT[t_step]:.0f}s.jpg',bbox_inches='tight',)

plt.show()

In [ ]:
t_step = 220

scan_speed = r'30 $\mathbf{\mu m/s}$'
power = r"25 $\mathbf{mW}$"

laser_speed = 30
scaling_factor = 401/1000 # scaling factor for plotting figure
laser_act_pos = 125 + timeS[t_step]*laser_speed  # Laser actual position in true dimension
laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure

x = np.arange(time_steps-1)

fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
ax1.set_facecolor((0, 0, 0, 0))

hmap1 = ax1.imshow(temp[:,:,t_step], cmap='Spectral_r', vmin=300, aspect=0.5,  interpolation='quadric')

ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
ax1.spines[:].set_linewidth(4)
ax1.spines[:].set_color('k')

if timeS[t_step] <= 25:
    ax1.arrow(laser_pos,-75, 0, 71,  width = 4, color='red', length_includes_head=True, clip_on=False)
    ax1.text(laser_pos-15,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
    if laser_act_pos < 500:
        ax1.set_title(f't = {timeS[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')
        ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
    else:
        ax1.set_title(f't = {timeS[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
        ax1.text(laser_pos-55, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos-55, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
else:
    ax1.set_title(f't = {timeS[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
    ax1.text(350, -40, "LASER", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
    ax1.text(350, -20, "OFF", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
    ax1.arrow(400, -10, -50, 0, fill=False, hatch='x', overhang=1, linestyle='--',  width = 3, color='blue', alpha=1, length_includes_head=True, clip_on=False)
    ax1.arrow(350, -75, 0, 71,  width = 4, color='none', length_includes_head=True, clip_on=False) # to make the figure height same after laser is turned off


################## Meltpool and Isotherm ################
cmap = plt.get_cmap('Wistia')
cmap.set_under('white', alpha=0) 
meltpool_1 = ax1.imshow(LIQ[:,:,t_step], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
ax1.arrow(leading_temperature[t_step], -14, 0, 10,  width = 7, head_width=7, color='lime', length_includes_head=True, clip_on=False)
ax1.text(leading_temperature[t_step]+3, -8, s=r'$\tau$', fontsize=25, weight = 'bold')
ax1.arrow(leading_LIQ[t_step], 11, 0, -10,  width = 7, head_width=7,  color='b', length_includes_head=True, clip_on=False)
ax1.text(leading_LIQ[t_step]+5, 20, s=r'$\mathfrak{D}$', fontsize=25, weight = 'bold')

################# Contour ##################
contour_levels = [1337]
contour = plt.contour(temp[:, :, t_step], levels=contour_levels, colors='orangered', linewidths=4,)
clabels = plt.clabel(contour, inline=True, fontsize=15, colors='k', fmt='%1.0f K')
for label in clabels: label.set_fontweight('bold')

############## Color Bar ###########
ax2 = fig.add_axes([0.92, 0.3, 0.05, 0.39]) 
ax2.tick_params(axis='both', labelcolor='black', labelsize=1, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
cbar = fig.colorbar(hmap1, cax=ax2, orientation='vertical')
cbar.ax.tick_params(labelsize=25, direction='in', length=0) 
text = Text(x=0.935, y=0.335, text=r'$\mathbf{\leftarrow}$   T [K]  $\mathbf{\rightarrow}$', fontsize=30, color='k', rotation=90, weight='bold')
fig.add_artist(text)
ax2.spines[:].set_linewidth(4)

if timeS[t_step] <= 25:
    cbar.set_ticks([np.min(temp[:,:,t_step])*1.06, (np.max(temp[:,:,t_step]))/2, np.max(temp[:,:,t_step])*0.96], labels=[f'{np.min(temp[:,:,t_step]):.0f}', f'{np.max(temp[:,:,t_step])/2 :.0f}', f'{np.max(temp[:,:,t_step])*0.95:.0f}'], weight='bold') 
else:
    cbar.set_ticks([np.min(temp[:,:,t_step]), (np.max(temp[:,:,t_step])+np.min(temp[:,:,t_step]))/2, np.max(temp[:,:,t_step])], labels=[f'{np.min(temp[:,:,t_step]):.0f}', f'{(np.max(temp[:,:,t_step])+np.min(temp[:,:,t_step]))/2 :.0f}', f'{np.max(temp[:,:,t_step]):.0f}'], weight='bold') 

################## Dimensions ###################
ax1.set_xlabel('$\mathbf{\longleftarrow}$                 1000 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
ax1.annotate(r'$\mathbf{\leftarrow}$ 250 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
             xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')

ax1.set_ylim(201, -1)
ax1.set_xlim(-1,401)

##############################################
##############################################
if laser_act_pos > 500:
    ax3 = fig.add_axes([0.18, 0.34, 0.2, 0.18], frameon=True)  # [left, bottom, width, height]

else:
    ax3 = fig.add_axes([0.63, 0.34, 0.2, 0.18], frameon=True)  # [left, bottom, width, height]
# ax3.set_facecolor((0, 0, 0, 0))
ax3.plot(leading_temperature[1:t_step], time_lead_lag[1:t_step], marker='o', c='lime',markersize=1, linewidth=2, label =r'$\tau$')
ax3.plot(leading_LIQ[1:t_step], time_lead_lag[1:t_step], marker='.', c='blue',markersize=1,linewidth=2, label =r'$\mathfrak{D}$')
ax3.set_ylabel("t [s]",  weight = 'bold', size=20, color='blue',labelpad=-1 )
ax3.set_xticks([55, 200, 380], labels=[ r'X$\mathbf{[\mu m]}\rightarrow$', f'{200*(1000/400):.0f}', f'1000'], size=15, color='white', weight='bold',)
ax3.xaxis.set_tick_params(direction='in', length=0) 
ax3.yaxis.set_tick_params(labelcolor='lime', labelsize=15,  direction='inout', length=8, width=2, color='k',)
for label in ax3.get_yticklabels(): label.set_fontweight('bold')
ax3.yaxis.set_major_locator(plt.MaxNLocator(4))
legend_1 = ax3.legend(loc=(0.41,0.57), ncol=2, columnspacing=0.5, prop={'weight':'bold', 'size':'30',}, handletextpad=0.1, handlelength=0.7,   frameon=False).get_frame().set_facecolor('none')

ax4 = ax3.twinx()
if t_step>common_position:
    ax4.plot(leading_LIQ[common_position:t_step],lag_time[common_position:t_step], linestyle='-',  c='red', linewidth=3, label =r'$\Delta t_{x}$') # positional_timelag
legend_2 = ax4.legend(loc=(0.74,-0.05),  prop={'weight':'bold', 'size':'20',}, handletextpad=0.1, handlelength=0.75,   frameon=False).get_frame().set_facecolor('none')
ax4.set_ylabel(r"$\mathbf{\delta t_{pos}}$ [s]",  weight = 'bold', size=20, color='red')
ax4.yaxis.set_tick_params(labelcolor='red', labelsize=17, direction='inout', length=8, width=2, color='k',)
ax4.yaxis.set_major_locator(plt.MaxNLocator(4))
for label in ax4.get_yticklabels(): label.set_fontweight('bold')
ax4.set_ylim(1.25,1.75)

ax3.spines[:].set_linewidth(2)
ax3.set_ylim([-2,28])
ax3.set_xlim([0,420])

plt.show()